# Data Import Notebook (`DataImport.ipynb`)

This notebook loads the three raw GoCab CSV files into a MySQL database (`gobest_cab2`).  
The focus here is **reliable, efficient importing**, not heavy cleaning or feature engineering.  
More advanced transformations are handled later in the ETL script (`etl_prefect.py`).

## 1. Setup and Database Connection

In this section we:

- Import core libraries: `pandas`, `numpy`, `sqlalchemy`, `pymysql`, and `tqdm` (for progress).
- Define the MySQL connection string, pointing to database `gobest_cab2`.
- Create an SQLAlchemy engine and test the connection using a simple `SELECT DATABASE();`.
- Optionally set some constants such as:
  - CSV file paths for `Driver`, `SafetyLabel`, and `SensorData`
  - Chunk size used when reading the large `SensorData` file.

This ensures that the notebook can talk to MySQL before any heavy work starts.

In [1]:
# ==============================
# CONNECT PYTHON TO MYSQL (SQLAlchemy 2.x FIXED)
# ==============================

from sqlalchemy import create_engine, text
import pymysql
import pandas as pd

# ---- Change these to your own credentials ----
username = "root"          # your MySQL username
password = "48835674#Ryleesucks" # your MySQL password
host = "localhost"         # usually localhost
database = "gobest_cab"    # your database name

# ---- Create connection string ----
connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"

# ---- Create SQLAlchemy engine ----
engine = create_engine(
    connection_string,
    pool_pre_ping=True,
    connect_args={"local_infile": 1}   # enable fast loader
)
# ---- Test connection ----
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT DATABASE();"))
        print("Connected to MySQL database:", result.scalar())
except Exception as e:
    print("Connection failed:", e)


Connected to MySQL database: gobest_cab


## 2. Quick Look at Raw CSV Files

Here we do light inspection of the raw files to understand their structure:

- Use `pd.read_csv(..., nrows=5)` to preview column names and example rows.
- Confirm that:
  - `Driver` contains one row per driver (id, name, DOB, rating, etc.).
  - `SafetyLabel` contains one row per trip (`bookingID`, `driver_id`, `label`).
  - `SensorData` contains second-by-second readings (`bookingID`, `second`, speed, accel, gyro, etc.).
- Check approximate file sizes and row counts (e.g. `len(pd.read_csv(..., usecols=['bookingID']))` or `wc -l` style checks).

This step is mainly for sanity checking that the files are what we expect.

In [2]:
# ==============================
# LOAD DRIVER CSV INTO PYTHON
# ==============================

import pandas as pd

# Replace with the actual path to your CSV
driver_csv_path = "drivers_dataset.csv"  

df_driver = pd.read_csv(driver_csv_path)

print("Driver CSV loaded successfully!")
df_driver.head()


Driver CSV loaded successfully!


,driver_id,driver_name,data_of_birth,No_of_Years_driving_exp,gender,car_brand,car_model_year,driver_rating
0,1,Sinclair Birmingham,10/17/1982,10,Male,Audi,2010,3.8
1,2,Juline Faulks,11/30/1977,14,Female,BMW,2000,2.8
2,3,Germayne Stit,9/18/1976,13,Male,Mercedes-Benz,1999,3.8
3,4,Athena Coudray,10/24/1982,16,Female,Lexus,2009,2.2
4,5,Batholomew Hiscoke,9/11/1978,20,Male,BMW,2012,2.7


## 3. Preparing MySQL Tables

Before importing, we make sure the target tables exist and are ready:

- Create the three base tables (`Driver`, `SafetyLabel`, `SensorData`) following the ERD:
  - `Driver(id, name, date_of_birth, gender, car_make, car_model_year, rating, no_of_years_driving_exp, …)`
  - `SafetyLabel(bookingID, driver_id, label, …)`
  - `SensorData(bookingID, second, accuracy, speed, acceleration_x, acceleration_y, acceleration_z, gyro_x, gyro_y, gyro_z, …)`
- If the tables already exist from previous runs, we either:
  - Truncate them, or
  - Drop and recreate them.

At this stage the schema mirrors the original design; we are not yet adding derived tables such as `TripFeatures` or `DriverSummary`.

In [3]:
# ==============================
# RENAME COLUMNS TO MATCH SQL TABLE
# ==============================

df_driver = df_driver.rename(columns={
    "driver_id": "id",
    "driver_name": "name",
    "data_of_birth": "date_of_birth",
    "No_of_Years_driving_exp": "no_of_years_driving_exp",
    "car_brand": "car_brand",
    "driver_rating": "rating"
})

df_driver.head()


,id,name,date_of_birth,no_of_years_driving_exp,gender,car_brand,car_model_year,rating
0,1,Sinclair Birmingham,10/17/1982,10,Male,Audi,2010,3.8
1,2,Juline Faulks,11/30/1977,14,Female,BMW,2000,2.8
2,3,Germayne Stit,9/18/1976,13,Male,Mercedes-Benz,1999,3.8
3,4,Athena Coudray,10/24/1982,16,Female,Lexus,2009,2.2
4,5,Batholomew Hiscoke,9/11/1978,20,Male,BMW,2012,2.7


## 4. Importing the `Driver` Table

Steps for loading the driver data:

1. Read the full `drivers_dataset.csv` with `pandas.read_csv`.
2. Perform minimal cleaning / type handling:
   - Parse `date_of_birth` as `datetime`.
   - Ensure numeric columns such as `rating` and `no_of_years_driving_exp` are cast to the correct types.
3. Write the DataFrame into MySQL using `df.to_sql("Driver", engine, if_exists="append", index=False)`.

After this, we have one row per driver in the `Driver` table, ready for later transformation in the ETL flow.

In [4]:
# ==============================
# FIX DATE FORMAT
# ==============================

df_driver["date_of_birth"] = pd.to_datetime(
    df_driver["date_of_birth"], 
    format="%m/%d/%Y"
)


In [5]:
# ==============================
# INSERT DRIVER DATA INTO MYSQL
# ==============================

df_driver.to_sql(
    name="Driver",
    con=engine,
    if_exists="append",
    index=False
)

print("Driver table successfully imported into MySQL!")


Driver table successfully imported into MySQL!


C:\Users\User\AppData\Local\Temp\ipykernel_16184\1697236138.py:5: UserWarning: The provided table name 'Driver' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_driver.to_sql(


## 5. Importing the `SafetyLabel` Table

Steps for loading trip labels:

1. Read `safety_status_dataset.csv` using `pandas`.
2. Cast `label` to integer and check that values are either `0` (safe) or `1` (dangerous).
3. Insert into MySQL:
   - `df.to_sql("SafetyLabel", engine, if_exists="append", index=False)`.

We now have a `SafetyLabel` table that links `bookingID` to `driver_id` and the binary safety label.

In [6]:
import pandas as pd

df_safety = pd.read_csv("safety_status_dataset.csv")
df_safety.head()


,bookingID,driver_id,label
0,111669149733,229,0
1,335007449205,312,1
2,171798691856,264,0
3,1520418422900,113,0
4,798863917116,392,0


In [7]:
df_safety.to_sql(
    name="safetylabel",
    con=engine,
    if_exists="append",
    index=False,
    chunksize=5000,
    method="multi"
)

print("Safety labels inserted successfully!")


Safety labels inserted successfully!


## 6. Efficient Chunked Import for `SensorData`

`SensorData` is the largest file and cannot be loaded safely in one shot.  
This section implements a **chunked bulk load** with basic validation:

### 6.1 Temporarily Relax Constraints

To speed up inserts:

- Drop the foreign key on `SensorData.bookingID` that points to `SafetyLabel.bookingID`.
- Drop the composite primary key (`bookingID, second`) on `SensorData`.
- This avoids constraint checks on every single row during the bulk insert.

### 6.2 Chunked Reading and Insertion

We then loop over the file in chunks, e.g. `chunk_size = 200000` or `500000`:

1. Use `pd.read_csv(..., chunksize=chunk_size)` to stream the file.
2. For each chunk:
   - Replace `NaN` with `None` for fields where MySQL does not accept `NaN`.
   - Ensure numeric columns (`speed`, `accuracy`, accel/gyro) are of numeric dtype.
   - Insert into MySQL with `chunk_df.to_sql("SensorData", engine, if_exists="append", index=False)`.
   - Print progress messages such as  
     *“Inserted rows 0–200,000 … 200,000–400,000 …”* so we can monitor the import.

The screenshot showing:
> `STEP 4 – Inserting 7,346,552 rows into SensorData…`
comes from this loop and confirms that all chunks were processed successfully.

### 6.3 Re-apply Constraints

After the bulk insert finishes:

- Recreate the primary key on (`bookingID`, `second`) for `SensorData`.
- Recreate the foreign key `SensorData.bookingID → SafetyLabel.bookingID`.

This restores relational integrity once the heavy loading work is done.

In [8]:
# ==============================
# LOAD ALL SENSOR CSV FILES INTO PYTHON
# ==============================

import pandas as pd
import glob

# Change to the folder where your sensor CSV files are stored
sensor_folder_path = "sensor_dataset_features/*.csv"

sensor_files = glob.glob(sensor_folder_path)

print("Number of sensor CSV files found:", len(sensor_files))

# Load all files into a single DataFrame
df_sensor_list = []

for file in sensor_files:
    print("Loading:", file)
    df = pd.read_csv(file)
    df_sensor_list.append(df)

# Merge all parts into one DataFrame
df_sensor = pd.concat(df_sensor_list, ignore_index=True)

print("Total sensor rows:", len(df_sensor))
df_sensor.head()


Number of sensor CSV files found: 5
Loading: sensor_dataset_features\0_Sensor_DataSet_features_part.csv
Loading: sensor_dataset_features\1-Sensor_DataSet_features_part.csv
Loading: sensor_dataset_features\2-Sensor_DataSet_features_part.csv
Loading: sensor_dataset_features\3_Sensor_DataSet_features_part.csv
Loading: sensor_dataset_features\4_Sensor_DataSet_features_part.csv
Total sensor rows: 7469656


,Unnamed: 0,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000
1,1,274877907034,9.293,17.0,0.032775,8.659933,4.737300,0.024629,0.004028,-0.010858,257.0,0.190000
2,2,884763263056,3.000,NaN,1.139675,9.545974,1.951334,-0.006899,-0.015080,0.001122,973.0,0.667059
3,3,1073741824054,3.900,126.0,3.871542,10.386364,-0.136474,0.001344,-0.339601,-0.017956,902.0,7.913285
4,4,1056561954943,3.900,50.0,-0.112882,10.550960,-1.560110,0.130568,-0.061697,0.161530,820.0,20.419409


In [9]:
# ==============================
# FINAL SENSOR CLEANING BLOCK (UPDATED)
# ==============================

# 1. Drop unwanted index columns
df_sensor = df_sensor.loc[:, ~df_sensor.columns.str.contains('^Unnamed')]

# 2. Rename incorrectly capitalized columns
df_sensor = df_sensor.rename(columns={
    "Accuracy": "accuracy",
    "Bearing": "bearing",
    "Speed": "speed"
})

# 3. Fix bookingID type
df_sensor["bookingID"] = df_sensor["bookingID"].astype(str)

# 4. Remove rows with missing PRIMARY KEY components
df_sensor = df_sensor.dropna(subset=["bookingID", "second"])

# 5. Ensure second is integer
df_sensor["second"] = df_sensor["second"].astype(int)

df_sensor.head()


,bookingID,accuracy,bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,speed
0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362,0.000000
1,274877907034,9.293,17.0,0.032775,8.659933,4.737300,0.024629,0.004028,-0.010858,257,0.190000
2,884763263056,3.000,NaN,1.139675,9.545974,1.951334,-0.006899,-0.015080,0.001122,973,0.667059
3,1073741824054,3.900,126.0,3.871542,10.386364,-0.136474,0.001344,-0.339601,-0.017956,902,7.913285
4,1056561954943,3.900,50.0,-0.112882,10.550960,-1.560110,0.130568,-0.061697,0.161530,820,20.419409


In [10]:
from sqlalchemy import text
from sqlalchemy.exc import IntegrityError
import pandas as pd

# ==========================================================
# AUTOMATED FAST INSERT INTO SensorData
# (DROP FK + PK → INSERT → RE-ADD PK + FK)
# ==========================================================

# ----------------------------------------------------------
# STEP 1 — DROP ANY FOREIGN KEYS DEFINED ON SensorData
# ----------------------------------------------------------

print("STEP 1 – Dropping foreign keys defined on SensorData...")

with engine.begin() as conn:
    fk_rows = conn.execute(text("""
        SELECT CONSTRAINT_NAME
        FROM information_schema.TABLE_CONSTRAINTS
        WHERE TABLE_SCHEMA = DATABASE()
          AND TABLE_NAME = 'SensorData'
          AND CONSTRAINT_TYPE = 'FOREIGN KEY';
    """)).fetchall()

    if fk_rows:
        for (constraint_name,) in fk_rows:
            print(f"  → Dropping FK {constraint_name} on SensorData...")
            conn.execute(text(f"""
                ALTER TABLE SensorData
                DROP FOREIGN KEY `{constraint_name}`;
            """))
        print("✔ All FKs on SensorData dropped.")
    else:
        print("  (No FKs defined on SensorData.)")

# ----------------------------------------------------------
# STEP 2 — DROP PRIMARY KEY ON SensorData
# ----------------------------------------------------------

print("\nSTEP 2 – Dropping PRIMARY KEY (bookingID, second) on SensorData...")

with engine.begin() as conn:
    try:
        conn.execute(text("ALTER TABLE SensorData DROP PRIMARY KEY;"))
        print("✔ Primary key dropped.")
    except Exception as e:
        print("⚠ Warning while dropping PK:", e)
        print("   (PK may already be dropped; if not, check for remaining FKs.)")

# ----------------------------------------------------------
# STEP 3 — CLEAN NaN → None IN df_sensor
# ----------------------------------------------------------

df_sensor = df_sensor.astype(object).where(pd.notnull(df_sensor), None)
print("\nSTEP 3 – Cleaned NaN → None in df_sensor.")

# ----------------------------------------------------------
# STEP 4 — FAST BATCH INSERT INTO SensorData
# ----------------------------------------------------------

def fast_insert(df, table_name, batch_size=200000):
    total = len(df)
    print(f"\nSTEP 4 – Inserting {total:,} rows into {table_name}...")

    columns = ", ".join(df.columns)
    placeholders = ", ".join([f":{col}" for col in df.columns])
    insert_sql = text(f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})")

    with engine.begin() as conn:
        for start in range(0, total, batch_size):
            end = min(start + batch_size, total)
            chunk = df.iloc[start:end]
            data = chunk.to_dict(orient="records")
            conn.execute(insert_sql, data)
            print(f"   → Inserted rows {start:,} to {end:,}")

    print("\n✔ INSERT COMPLETE — All SensorData rows loaded!")

fast_insert(df_sensor, "SensorData", batch_size=200000)

# ----------------------------------------------------------
# STEP 5 — RE-ADD PRIMARY KEY ON SensorData
# ----------------------------------------------------------

print("\nSTEP 5 – Re-adding PRIMARY KEY (bookingID, second) on SensorData...")

with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE SensorData
        ADD PRIMARY KEY (bookingID, second);
    """))

print("✔ Primary key restored.")

# ----------------------------------------------------------
# STEP 6 — RE-ADD FK: SensorData.bookingID → SafetyLabel.bookingID
# ----------------------------------------------------------

print("\nSTEP 6 – Adding FK SensorData.bookingID -> SafetyLabel.bookingID...")

try:
    with engine.begin() as conn:
        conn.execute(text("""
            ALTER TABLE SensorData
            ADD CONSTRAINT fk_sensordata_booking
            FOREIGN KEY (bookingID)
            REFERENCES SafetyLabel(bookingID);
        """))
    print("✔ Foreign key fk_sensordata_booking added.")
except IntegrityError as e:
    print("❌ IntegrityError when adding FK fk_sensordata_booking.")
    print("   This means some SensorData.bookingID values do NOT exist in SafetyLabel.")
    
    # Optional quick diagnostics:
    with engine.begin() as conn:
        # count orphans
        orphan_count = conn.execute(text("""
            SELECT COUNT(*) 
            FROM SensorData s
            LEFT JOIN SafetyLabel sl USING (bookingID)
            WHERE sl.bookingID IS NULL;
        """)).scalar()
        print(f"   → Orphan SensorData rows (no matching SafetyLabel): {orphan_count}")

        if orphan_count > 0:
            sample_orphans = conn.execute(text("""
                SELECT DISTINCT s.bookingID
                FROM SensorData s
                LEFT JOIN SafetyLabel sl USING (bookingID)
                WHERE sl.bookingID IS NULL
                LIMIT 10;
            """)).fetchall()
            print("   → Sample orphan bookingIDs:", [row[0] for row in sample_orphans])

    print("   Fix options:")
    print("   - Reload SafetyLabel with all bookingIDs, OR")
    print("   - Delete orphan rows from SensorData before re-adding the FK.")
    print("   Error detail:", e)

print("\n🎉 ALL STEPS COMPLETED (or failed gracefully with diagnostics).")


STEP 1 – Dropping foreign keys defined on SensorData...
  → Dropping FK sensordata_ibfk_1 on SensorData...
✔ All FKs on SensorData dropped.

STEP 2 – Dropping PRIMARY KEY (bookingID, second) on SensorData...
✔ Primary key dropped.

STEP 3 – Cleaned NaN → None in df_sensor.

STEP 4 – Inserting 7,346,552 rows into SensorData...
   → Inserted rows 0 to 200,000
   → Inserted rows 200,000 to 400,000
   → Inserted rows 400,000 to 600,000
   → Inserted rows 600,000 to 800,000
   → Inserted rows 800,000 to 1,000,000
   → Inserted rows 1,000,000 to 1,200,000
   → Inserted rows 1,200,000 to 1,400,000
   → Inserted rows 1,400,000 to 1,600,000
   → Inserted rows 1,600,000 to 1,800,000
   → Inserted rows 1,800,000 to 2,000,000
   → Inserted rows 2,000,000 to 2,200,000
   → Inserted rows 2,200,000 to 2,400,000
   → Inserted rows 2,400,000 to 2,600,000
   → Inserted rows 2,600,000 to 2,800,000
   → Inserted rows 2,800,000 to 3,000,000
   → Inserted rows 3,000,000 to 3,200,000
   → Inserted rows 3,200